In [1]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://developers.google.com"
url1 = "https://api.sportsdata.io/api/nba/fantasy/json/CurrentSeason"
url2 = "http://api.rottentomatoes.com/api/public/v1.0/lists/movies/box_office.json"

In [3]:
response = requests.get(url)
response.status_code

200

In [4]:
response = requests.get(url1)
response.status_code

401

In [5]:
response = requests.get(url2)
response.status_code

403

# Skyscanner

In [3]:
import requests

In [16]:
# flights from where to where on what date
url = 'https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/UK/GBP/en-GB/SFO-sky/NYCA-sky/2020-12-12'
params = {'inboundpartialdate':'2020-12-12'}
headers = {
    'x-rapidapi-host': 'skyscanner-skyscanner-flight-search-v1.p.rapidapi.com',
    'x-rapidapi-key': 'f440cd47d9msh1242e8fec2b54bcp1114b0jsn2208616b850d'}
#url -- url of the API
#params -- parameter of the date
#header-- you get that from the API web


response = requests.get(url,headers = headers, params = params)

response.json()

{'Quotes': [{'QuoteId': 1,
   'MinPrice': 83,
   'Direct': True,
   'OutboundLeg': {'CarrierIds': [851],
    'OriginId': 81727,
    'DestinationId': 50290,
    'DepartureDate': '2020-12-12T00:00:00'},
   'QuoteDateTime': '2020-12-01T09:33:00'}],
 'Carriers': [{'CarrierId': 851, 'Name': 'Alaska Airlines'}],
 'Places': [{'Name': 'New York Newark',
   'Type': 'Station',
   'PlaceId': 50290,
   'IataCode': 'EWR',
   'SkyscannerCode': 'EWR',
   'CityName': 'New York',
   'CityId': 'NYCA',
   'CountryName': 'United States'},
  {'Name': 'San Francisco International',
   'Type': 'Station',
   'PlaceId': 81727,
   'IataCode': 'SFO',
   'SkyscannerCode': 'SFO',
   'CityName': 'San Francisco',
   'CityId': 'SFOA',
   'CountryName': 'United States'}],
 'Currencies': [{'Code': 'GBP',
   'Symbol': '£',
   'ThousandsSeparator': ',',
   'DecimalSeparator': '.',
   'SymbolOnLeft': True,
   'SpaceBetweenAmountAndSymbol': False,
   'RoundingCoefficient': 0,
   'DecimalDigits': 2}]}

In [17]:
# Turning the information into data frame
import pandas as pd
import json
# We import json because the API outputs on the json

In [18]:
# We normalize the information
pd.json_normalize(response.json())

,Quotes,Carriers,Places,Currencies
0,"[{'QuoteId': 1, 'MinPrice': 83, 'Direct': True...","[{'CarrierId': 851, 'Name': 'Alaska Airlines'}]","[{'Name': 'New York Newark', 'Type': 'Station'...","[{'Code': 'GBP', 'Symbol': '£', 'ThousandsSepa..."


In [19]:
# imported to a Data Frame
quotes = pd.DataFrame(pd.json_normalize(response.json())['Quotes'][0])
quotes

,QuoteId,MinPrice,Direct,OutboundLeg,QuoteDateTime
0,1,83,True,"{'CarrierIds': [851], 'OriginId': 81727, 'Dest...",2020-12-01T09:33:00


In [22]:
flights = pd.DataFrame(pd.DataFrame(pd.json_normalize(response.json())['Quotes'][0])['OutboundLeg'][0])
flights

,CarrierIds,OriginId,DestinationId,DepartureDate
0,851,81727,50290,2020-12-12T00:00:00


In [21]:
quotes = pd.DataFrame(pd.json_normalize(response.json())['Carriers'][0])
quotes
# We index 0, because we want to get the first rows

,CarrierId,Name
0,851,Alaska Airlines


In [23]:
places = pd.DataFrame(pd.json_normalize(response.json())['Places'][0])
places

,Name,Type,PlaceId,IataCode,SkyscannerCode,CityName,CityId,CountryName
0,New York Newark,Station,50290,EWR,EWR,New York,NYCA,United States
1,San Francisco International,Station,81727,SFO,SFO,San Francisco,SFOA,United States


In [24]:
# We create a function that returns the lowest price 

def lowest_price(origin, destination, start, end):
    flights = flight_days(origin, destination, start, end)
    prices = {date:min([quote["MinPrice"] for quote in flights[date]["Quotes"]]) for date in flights for quote in flights[date]["Quotes"]}
    return min(prices, key = prices.get)